<a href="https://colab.research.google.com/github/Rafnarok335/Spam_Classification-/blob/main/bert(rafi)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Installing tensorflow_text
!pip3 install tensorflow_text

  Using cached tensorflow_text-2.8.1-cp39-cp39-win_amd64.whl (2.5 MB)


In [ ]:
!pip3 install tensorflow


##Importing libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd
##reading the dataset and doing some preprocessing of the dataset
df = pd.read_csv("spam.csv")
df.head(5)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.groupby('Category').describe()


Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [ ]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
747/4825

0.15481865284974095

In [ ]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [ ]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [ ]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [ ]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [ ]:
"""adding another column for the dataset named spam if the category has value spam the row value of spam will be 1 otherwise 1
used a lamda function which is a function which takes X as an argument and gives a certain condition to execute a simple code """

df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
415,spam,100 dating service cal;l 09064012103 box334sk38ch,1
1199,ham,Al he does is moan at me if n e thin goes wron...,0
3819,spam,74355 XMAS iscoming & ur awarded either £500 C...,1
4297,spam,Please CALL 08712402578 immediately as there i...,1
4076,ham,"God picked up a flower and dippeditinaDEW, lov...",0


In [ ]:
###Splitting the dataset here we splitting the message column as x and span as y doing stratification for balancing the train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
X_train.head(4)

5445    And that's fine, I got enough bud to last most...
1194    Sex up ur mobile with a FREE sexy pic of Jorda...
1714    Hard LIVE 121 chat just 60p/min. Choose your g...
2508                                               Yup...
Name: Message, dtype: object

In [ ]:
##To use bert we can use URLs of the pretrained model and encoder of the model from tensorflow hub / we can download the model from the hub
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") ##this is the pretrained model of bert
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4") ##this is the encoder of the bert

In [ ]:
"""This function is just an embedding function which takes sentences as input and returns an embedding vector for the sentence """
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

In [ ]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity ##importing cosine similarity for reviewing similarity between words/ sentences
cosine_similarity([e[0]],[e[1]])

array([[0.99110883]], dtype=float32)

In [ ]:
cosine_similarity([e[0]],[e[3]]) ##similarity betwwen banana and jeff bezos

array([[0.8470385]], dtype=float32)

In [ ]:
cosine_similarity([e[3]],[e[4]]) ##similarity between jeff bezox and elon musk

array([[0.98720354]], dtype=float32)

In [ ]:
# Bert layers
"""So bert is a funtional model; previously we have used sequential model for creating networks or layers.The difference between functional and sequential is that
in sequentail we just added layers individully but in functional model we add a layer and in the second layer we pass the first layer as a parameter in the second
layer we do this one by one"""
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')##input layer
preprocessed_text = bert_preprocess(text_input) ##getting an embedded vector 
outputs = bert_encoder(preprocessed_text) ##this is the encoder 

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output']) ##to prevent overfitting adding a dropout layer,output layer is passed as parameter
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l) ##output layer , the l varaible is passed as a parameter 

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l]) ##in functional model we have only input and output parameters

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
len(X_train)

1120

In [ ]:
##Evalution of the model using some popular metrices
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
##compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10) ##model trainning with 10 epochs

Epoch 1/10
35/35 [==============================] - 172s 5s/step - loss: 0.6516 - accuracy: 0.6187 - precision: 0.6169 - recall: 0.6268
Epoch 2/10
35/35 [==============================] - 164s 5s/step - loss: 0.5249 - accuracy: 0.7973 - precision: 0.8011 - recall: 0.7911
Epoch 3/10
35/35 [==============================] - 164s 5s/step - loss: 0.4508 - accuracy: 0.8429 - precision: 0.8221 - recall: 0.8750
Epoch 4/10
35/35 [==============================] - 166s 5s/step - loss: 0.4001 - accuracy: 0.8687 - precision: 0.8604 - recall: 0.8804
Epoch 5/10
35/35 [==============================] - 163s 5s/step - loss: 0.3688 - accuracy: 0.8759 - precision: 0.8598 - recall: 0.8982
Epoch 6/10
35/35 [==============================] - 172s 5s/step - loss: 0.3397 - accuracy: 0.8804 - precision: 0.8586 - recall: 0.9107
Epoch 7/10
35/35 [==============================] - 156s 4s/step - loss: 0.3236 - accuracy: 0.8955 - precision: 0.8893 - recall: 0.9036
Epoch 8/10
35/35 [==============================

In [ ]:
model.evaluate(X_test, y_test) ##evaluation of the model

12/12 [==============================] - 53s 4s/step - loss: 0.2727 - accuracy: 0.9278 - precision: 0.9301 - recall: 0.9251


[0.27274131774902344,
 0.9278075098991394,
 0.9301075339317322,
 0.9251337051391602]

In [ ]:
y_predicted = model.predict(X_test) ##model prediction
y_predicted = y_predicted.flatten() ##making it 1D

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted) 
cm

In [ ]:
##Plots to visualize the results

from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
""" actually these are emails of 5 index the return arrays are a softmax output the value greater than >0.5 is considered as spam email otherwise ham"""
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)